In [1]:
import requests
import pandas as pd
import json

### Utility function to get a JSON object response

In [2]:
def getJsonObject(url):

    headers = {'Content-Type': 'application/json'}
    response = requests.get(url, headers=headers)
    jsonObject = {}
    if response.status_code == 200:
        jsonObject = json.loads(response.content.decode('utf-8'))
    return jsonObject

### Function to get growth rates of a state

In [3]:
def getGrowth(state, confirmed, dates):

    growth_difference = []
    
    #start of first lockdown in India
    lockdown1 = '2020-03-25'
    #start of second lockdown in India
    lockdown2 = '2020-04-15'

    g1, g2, current_growth = 0, 0, 0
    
    #growth rate has been defined as
    # (number of confirmed cases on day i) / (number of confirmed cases on day i - 1) 

    for i in range(1, len(confirmed)):
        growth_difference.append(confirmed[i] / confirmed[i-1])
        if i != len(confirmed) - 1 and dates[i + 1] == lockdown1:
            g1 = sum(growth_difference)/len(growth_difference)
        elif i != len(confirmed) - 1 and dates[i + 1] == lockdown2:
            g2 = sum(growth_difference)/len(growth_difference)

    current_growth = sum(growth_difference)/len(growth_difference)

    df = pd.DataFrame(data=[[round(g1, 3), round(g2, 3), round(current_growth, 3), state]], columns=[
                      'g1', 'g2', 'current', 'state'])
    return df

## Growth Rates for India as a whole

In [4]:
# The api url from which the time series data is taken for India as a whole 
url = 'https://api.covid19india.org/data.json'
jsonObject = getJsonObject(url)

time_series = jsonObject['cases_time_series']
print(time_series[0])

{'dailyconfirmed': '1', 'dailydeceased': '0', 'dailyrecovered': '0', 'date': '30 January ', 'totalconfirmed': '1', 'totaldeceased': '0', 'totalrecovered': '0'}


In [5]:
months = {
    'january': '01',
    'february': '02',
    'march': '03',
    'april': '04',
    'may': '05',
    'june': '06',
    'july': '07',
    'august': '08',
    'september': '09',
    'october': '10',
    'november': '11',
    'december': '12'
}

In [6]:
confirmed = []
dates = []

In [7]:
for i in time_series:
    c = int(i['totalconfirmed'])
    confirmed.append(c)
    d = i['date'].strip().split(' ')
    date = '2020-' + months[d[1].lower()] + '-' + d[0]
    dates.append(date)

In [8]:
df = pd.DataFrame(data=None, columns=['g1', 'g2', 'current', 'state'])

df = df.append(getGrowth('India', confirmed, dates))
print('Growth before lockdown on 25th March :', df['g1'][0])
print('Growth until second lockdown on 14th April :', df['g2'][0])
print('Current growth :',df['current'][0])

Growth before lockdown on 25th March : 1.173
Growth until second lockdown on 14th April : 1.168
Current growth : 1.145


## Growth Rates for individual states

In [9]:
# The api url which is used to form a mapping between state name and state code given in the API
url = 'https://api.covid19india.org/v2/state_district_wise.json'
jsonObject = getJsonObject(url)

In [10]:
state_mapping = {}
state_truth = {}
state_growth = {}

for i in jsonObject:
    state = i['state']
    code = i['statecode'].lower()
    state_mapping[code] = state
    state_truth[code] = False
    state_growth[state] = {
        'dates': [],
        'cases': []
    }

In [11]:
# The api url from which the time series data is taken for each individual state
url = 'https://api.covid19india.org/states_daily.json'
jsonObject = getJsonObject(url)
states_daily = jsonObject['states_daily']
states_daily[0]

{'an': '0',
 'ap': '1',
 'ar': '0',
 'as': '0',
 'br': '0',
 'ch': '0',
 'ct': '0',
 'date': '14-Mar-20',
 'dd': '0',
 'dl': '7',
 'dn': '0',
 'ga': '0',
 'gj': '0',
 'hp': '0',
 'hr': '14',
 'jh': '0',
 'jk': '2',
 'ka': '6',
 'kl': '19',
 'la': '0',
 'ld': '0',
 'mh': '14',
 'ml': '0',
 'mn': '0',
 'mp': '0',
 'mz': '0',
 'nl': '0',
 'or': '0',
 'pb': '1',
 'py': '0',
 'rj': '3',
 'sk': '0',
 'status': 'Confirmed',
 'tg': '1',
 'tn': '1',
 'tr': '0',
 'tt': '81',
 'up': '12',
 'ut': '0',
 'wb': '0'}

### Utiltity function for formatting Dates

In [12]:
def getDate(s):
    parts = s.split('-')
    date = '2020-' + month_abbr[parts[1]] + '-' + parts[0]
    return date

In [13]:
month_abbr = {
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
}

In [14]:
for i in states_daily:
    if i['status'] == 'Confirmed':
        date = getDate(i['date'])
        del i['date']
        del i['status']
        del i['tt']
        for state in i.keys():
            try:
                if int(i[state]) > 0 or state_truth[state]:
                    state_truth[state] = True
                    sf = state_mapping[state]
                    state_growth[sf]['dates'].append(date)
                    if len(state_growth[sf]['cases']) != 0:
                        state_growth[sf]['cases'].append(
                            int(i[state]) + state_growth[sf]['cases'][-1])
                    else:
                        state_growth[sf]['cases'].append(int(i[state]))
            except ValueError:
                continue
            except KeyError:
                continue
    else:
        continue

In [15]:
#del state_growth['Dadra and Nagar Haveli and Daman and Diu']

In [16]:
for state in state_growth.keys():
    print(state, ':\n')
    confirmed = state_growth[state]['cases']
    dates = state_growth[state]['dates']
    df = getGrowth(state,confirmed, dates)
    print('Growth before lockdown on 25th March :', df['g1'][0])
    print('Growth until second lockdown on 14th April :', df['g2'][0])
    print('Current growth :',df['current'][0],'\n')

Andaman and Nicobar Islands :

Growth before lockdown on 25th March : 0
Growth until second lockdown on 14th April : 1.301
Current growth : 1.164 

Andhra Pradesh :

Growth before lockdown on 25th March : 1.318
Growth until second lockdown on 14th April : 1.272
Current growth : 1.181 

Arunachal Pradesh :

Growth before lockdown on 25th March : 0
Growth until second lockdown on 14th April : 1.0
Current growth : 1.0 

Assam :

Growth before lockdown on 25th March : 0
Growth until second lockdown on 14th April : 2.129
Current growth : 1.441 

Bihar :

Growth before lockdown on 25th March : 1.25
Growth until second lockdown on 14th April : 1.18
Current growth : 1.14 

Chandigarh :

Growth before lockdown on 25th March : 1.873
Growth until second lockdown on 14th April : 1.217
Current growth : 1.156 

Chhattisgarh :

Growth before lockdown on 25th March : 1.0
Growth until second lockdown on 14th April : 1.194
Current growth : 1.116 

Delhi :

Growth before lockdown on 25th March : 1.168
Gr